Part 2 for Metabolism Project

#1)

In [1]:
from pathlib import Path
from cobra.io import read_sbml_model
from cobra.flux_analysis import single_gene_deletion
import pandas as pd

In [3]:
sbml_path = Path("Met_Q2/ralsto_metexplore3.xml")

model = read_sbml_model(str(sbml_path))

Adding exchange reaction EX_hdcea_b with default bounds for boundary metabolite: hdcea_b.
Adding exchange reaction EX_RSc1944_b with default bounds for boundary metabolite: RSc1944_b.
Adding exchange reaction EX_3oochslac_b with default bounds for boundary metabolite: 3oochslac_b.
Adding exchange reaction EX_glu_D_b with default bounds for boundary metabolite: glu_D_b.
Adding exchange reaction EX_RipS4_b with default bounds for boundary metabolite: RipS4_b.
Adding exchange reaction EX_fecrm_b with default bounds for boundary metabolite: fecrm_b.
Adding exchange reaction EX_ni2_b with default bounds for boundary metabolite: ni2_b.
Adding exchange reaction EX_RipK_b with default bounds for boundary metabolite: RipK_b.
Adding exchange reaction EX_berb_b with default bounds for boundary metabolite: berb_b.
Adding exchange reaction EX_tartr_L_b with default bounds for boundary metabolite: tartr_L_b.
Adding exchange reaction EX_octa_b with default bounds for boundary metabolite: octa_b.
Addi

#2)

In [4]:
model

Name,_bc9d1403_3bf5_4994_8369_f3c28ce8995a
Memory address,1a99f53a0
Number of metabolites,2574
Number of reactions,3097
Number of genes,2219
Number of groups,253
Objective expression,0
Compartments,"boundary, extracellular, cytoplasm, periplasm"


As shown in the above table from the "model" variable, there are:

3097 Reactions
2574 Metabolites
2219 Genes

#3)

In [5]:
biomass_candidates = [r for r in model.reactions if "BIOMASS" in r.id.upper()]
[(r.id, r.name) for r in biomass_candidates]


[('BIOMASS', 'Biomass_formation')]

In [6]:
model.objective = "BIOMASS"

print("Objectif:", model.objective.expression)

Objectif: 1.0*BIOMASS - 1.0*BIOMASS_reverse_69053


#4)

In [7]:
len(model.exchanges)

453

In [8]:
for rxn in model.exchanges:
    rxn.lower_bound = 0.0

#5)

In [9]:
allowed_metabolites = [
    "h2o_b", "h_b", "k_b", "pi_b", "na1_b", "nh4_b", "so4_b", "mg2_b",
    "cl_b", "fe2_b", "fe3_b", "cobalt2_b", "mn2_b", "mobd_b",
    "o2_b", "co2_b"
]

In [10]:
for met in allowed_metabolites:
    rxn_id = f"EX_{met}"
    rxn = model.reactions.get_by_id(rxn_id)
    rxn.lower_bound = -1000.0
    rxn.upper_bound = 1000.0

#6)

Exchange reactions are those in which the cell interacts with the outside environment.  By setting the lower bound of all exchange reactions to 0, we are essentially closing the system to all imports or exports to or from the environment.  

Reopenning the reactions from the list explicitely stated in the previous question allows us to control exactly which molecules the cell is allowed to import or export to or from the environment.  This way, the cell will be able to interact with what we tell it is available in the environment and using cellular processes to access the other componants that are necessary for the system, instead of simply importing anything that it needs and exporting everything it doesn't need.

#7)

In [11]:
rxn = model.reactions.get_by_id("NGAME")
rxn.lower_bound = 8.39
rxn.upper_bound = 8.39

In [12]:
rxn = model.reactions.get_by_id("FEOXpp")
rxn.lower_bound = 0.0
rxn.upper_bound = 0.0

In [13]:
for rid in ["NGAME", "FEOXpp"]:
    r = model.reactions.get_by_id(rid)
    print(r.id, r.lower_bound, r.upper_bound)

NGAME 8.39 8.39
FEOXpp 0.0 0.0


#8)

In [14]:
#solution = model.optimize()
#print("Status:", solution.status)
#print("Growth rate (objective):", solution.objective_value)

##SolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.##

model.solver = "glpk"
solution = model.optimize()
print(solution.status, solution.objective_value)

infeasible 0.0


/opt/miniconda3/envs/cplex_env/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


The solution is "infeasible" or unsolvable because the cell isn't growing, it's in a state of maintenance.  By fixing the majority of the exchange reactions to 0, and then only allowing in some inorganic ions and gasses along with imposed restrictions on ATP demand with NGAME, the cell cannot be satisfied.  The cell has infeasable energy demands because it has no way to import the precursors necessary to produce any.  With the cell not able to meet the requirements, there is no optimal growth rate.

#9)

In [15]:
exp_flux = {
    "glu_L_b":   -8.25,
    "3ohpame_b":  1.5e-4,
    "EPS_b":      0.0062,
    "ptrc_b":     0.28,
    "Tek_28kd_b": 2.7e-4,
    "etle_b":     0.0129,
}

In [16]:
for met_id, v in exp_flux.items():
    met = model.metabolites.get_by_id(met_id)
    print("\n======================================================")
    print("Metabolite:", met.id, "-", met.name, "| target flux =", v)
    print("Reactions that include this metabolite:")

    for rxn in met.reactions:
        tag = ""
        if rxn.id.startswith("EX_"):
            tag = "[EXCHANGE]"
        print("  ", rxn.id, tag, "|", rxn.reaction)



Metabolite: glu_L_b - L_Glutamate | target flux = -8.25
Reactions that include this metabolite:
   EX_glu_L_e_ [EXCHANGE] | glu_L_e <=> glu_L_b
   EX_glu_L_b [EXCHANGE] | glu_L_b --> 

Metabolite: 3ohpame_b - 3_hydroxypalmitic_acid_methyl_ester | target flux = 0.00015
Reactions that include this metabolite:
   EX_3OHPAMES_e_ [EXCHANGE] | 3ohpame_e <=> 3ohpame_b
   EX_3ohpame_b [EXCHANGE] | 3ohpame_b --> 

Metabolite: EPS_b - Ralstonia_solanacearum_Exopolysaccharide | target flux = 0.0062
Reactions that include this metabolite:
   EX_EPS_b [EXCHANGE] | EPS_b --> 
   EX_EPS_e_ [EXCHANGE] | EPS_e <=> EPS_b

Metabolite: ptrc_b - Putrescine | target flux = 0.28
Reactions that include this metabolite:
   EX_ptrc_b [EXCHANGE] | ptrc_b --> 
   EX_ptrc_e_ [EXCHANGE] | ptrc_e <=> ptrc_b

Metabolite: Tek_28kd_b - Protein_Tek_28k_dalton_fragment | target flux = 0.00027
Reactions that include this metabolite:
   EX_Tek_e_ [EXCHANGE] | Tek_28kd_e <=> Tek_28kd_b
   EX_Tek_28kd_b [EXCHANGE] | Tek_28k

In [17]:
transport_rxn_ids = {
    "glu_L_b":   "EX_glu_L_b",
    "3ohpame_b": "EX_3ohpame_b",
    "EPS_b":     "EX_EPS_b",
    "ptrc_b":    "EX_ptrc_b",
    "Tek_28kd_b":"EX_Tek_28kd_b",
    "etle_b":    "EX_etle_b",
}

for met_id, rxn_id in transport_rxn_ids.items():
    v = exp_flux[met_id]
    rxn = model.reactions.get_by_id(rxn_id)
    rxn.lower_bound = float(v)
    rxn.upper_bound = float(v)
    print("Fixed", rxn.id, "to", v)


Fixed EX_glu_L_b to -8.25
Fixed EX_3ohpame_b to 0.00015
Fixed EX_EPS_b to 0.0062
Fixed EX_ptrc_b to 0.28
Fixed EX_Tek_28kd_b to 0.00027
Fixed EX_etle_b to 0.0129


In [18]:
solution = model.optimize()
print("Status:", solution.status)

if solution.status == "optimal":
    print("Growth rate:", solution.objective_value)
else:
    print("Infeasible -> no feasible steady-state flux distribution under these constraints.")


Status: optimal
Growth rate: 0.5119279935093651


After changing the constraints to include other metabolites that are necessary to meet the constraints, the cell become feasible and is able to create enough biomass to proliferate.  This means that we get a growth rate that is higher than 0, in this case .5119.  The initial growth rate of 0 was due to there being too many constraints on what the cell could import or export from the outside environment.

#10)

Target: .28

In [19]:
biomass = model.reactions.get_by_id("BIOMASS")
biomass.lower_bound = 0.28
biomass.upper_bound = 0.28

ng = model.reactions.get_by_id("NGAME")
ng.lower_bound = 0
ng.upper_bound = 10000

ngame = model.reactions.get_by_id("NGAME")
model.objective = ngame
model.objective.direction = "max"

sol = model.optimize()
print("Status:", sol.status)

if sol.status == "optimal":
    print("NGAME at growth=0.28:", sol.fluxes["NGAME"])
else:
    print("Infeasible: cannot achieve growth = 0.28 with current constraints.")


Status: optimal
NGAME at growth=0.28: 64.7583679616674


The value that gets closest to a flux of .28 is NGAME=64.758367.

This was found by making maximizing ngame the objective of the model and setting the biomass fixed at .28 (the goal of the question).  The model then solved for the NGAME that would produce this exact growth rate.

#11)

In [20]:
df = solution.fluxes.reset_index()
df.columns = ["Identifier", "cond1"]
df["Identifier"] = df["Identifier"].apply(lambda x: x if x.startswith("R_") else f"R_{x}")

df['cond1'] = df['cond1'].map(lambda x: f"{x:.12f}")

df.to_csv("/Users/donnysiii/Documents/Metabo/flux.txt", sep="\t", index=False)

![Flux distribution in glycolysis subnetwork of Ralstonia](cobrapy_images/graph(4).png)

#12)

In [21]:
del_res = single_gene_deletion(model)  # returns a DataFrame
del_res.head()

,ids,growth,status
0,{RSc2139},64.758368,optimal
1,{d0112},64.758368,optimal
2,{e0396},64.758368,optimal
3,{RSc1025},64.758368,optimal
4,{RSc3127},64.758368,optimal


In [22]:
del_res.columns

Index(['ids', 'growth', 'status'], dtype='object')

In [23]:
growth = float(sol.objective_value)
threshold = 0.01 * growth

growth_col = "growth" if "growth" in del_res.columns else "objective_value"

essential = del_res[
    (del_res["status"] != "optimal") | (del_res[growth_col] < threshold)
].copy()

print("Number of essential genes:", len(essential))
essential.head()

essential_genes = essential["ids"].tolist() if "ids" in essential.columns else essential.index.tolist()
essential_genes[:20], len(essential_genes)

Number of essential genes: 346


([{'RSc2720'},
  {'d0092'},
  {'RSc1172'},
  {'RS04383'},
  {'RSp1002'},
  {'RSc2557'},
  {'RSc2780'},
  {'RSc3322'},
  {'RSc0058'},
  {'d0093'},
  {'RSc1417'},
  {'RSc1019'},
  {'RSc0111'},
  {'RSc2191'},
  {'RS03938'},
  {'RSc1410'},
  {'d0134'},
  {'RSc2448'},
  {'RSc1072'},
  {'RSc2768'}],
 346)

In [24]:
essential_genes = [list(g)[0] for g in essential_genes]
essential_genes

['RSc2720',
 'd0092',
 'RSc1172',
 'RS04383',
 'RSp1002',
 'RSc2557',
 'RSc2780',
 'RSc3322',
 'RSc0058',
 'd0093',
 'RSc1417',
 'RSc1019',
 'RSc0111',
 'RSc2191',
 'RS03938',
 'RSc1410',
 'd0134',
 'RSc2448',
 'RSc1072',
 'RSc2768',
 'RS03460',
 'RSc1582',
 'RSc2435',
 'RSc1566',
 'RSc1619',
 'd0080',
 'd0127',
 'RSc1414',
 'RSc2900',
 'RSc0914',
 'RSp0624',
 'RSc2977',
 'RSc0093',
 'RSc2200',
 'RSc2461',
 'RSc0109',
 'RSc0712',
 'RS00236',
 'RSc2458',
 'RSc2847',
 'RSc0708',
 'RSc1028',
 'RSc2947',
 'RSc0688',
 'RSc0100',
 'd0055',
 'RSc2893',
 'RS03760',
 'RSc0666',
 'd0102',
 'RSc0523',
 'RSc2623',
 'RSc1862',
 'e0041',
 'RSc3445',
 'RSc2630',
 'RSc1319',
 'RSp1293',
 'RSc1232',
 'e0090',
 'RSc1328',
 'RSc1179',
 's0001',
 'RSc2423',
 'RSc0287',
 'RSc0686',
 'RS01509',
 'RSp0242',
 'RSp0020',
 'RSc2773',
 'RSc1381',
 'RSc0678',
 'RSc2905',
 'RSc1519',
 'RSc2952',
 'RSc2396',
 'RSc0685',
 'RSc2454',
 'RSc0713',
 'RSc0683',
 'RSc2628',
 'RS01162',
 'd0227',
 'RS02662',
 'RSc2953',
 '

In [25]:
genes = pd.DataFrame({
    "Identifier": essential_genes,
})

genes.to_csv("/Users/donnysiii/Documents/Metabo/essential_genes.tsv", sep="\t", index=False)
genes


,Identifier
0,RSc2720
1,d0092
2,RSc1172
3,RS04383
4,RSp1002
...,...
341,RSc1993
342,d0042
343,RSc0360
344,RSc2387


After mapping the essential genes, I sorted the result pathways by pvalue and found that that 10 most significant were lipopolysaccharide_biosynthesis, adenosylcobalamin_biosynthesis, arganine_biosynthesis, folate_biosynthesis, ubiquinone_biosynthesis, histidine_biosynthesis, isoprenoid_biosynthesis, purine_biosynthesis, peptidoglycan_biosynthesis, and flavin_biosynthesis.

Here are the statistics from the mapping:

![Mapping Statistics](cobrapy_images/q2_mappingStats.png)

The results, pathways sorted by pvalue:

![Mapping Statistics](cobrapy_images/q2_mappingPV.png)

Graph visualization, pathways sorted by pvalue:

![Mapping Statistics](cobrapy_images/q2_mappingGraph.png)